In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

In [2]:
x=tf.Variable(5,name="x")
y=tf.Variable(10,name="y")
f=x*x+y*y

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)
sess.close()

125


In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
    print(result)

125


In [5]:
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result=f.eval()
    print(result)

125


In [6]:
sess = tf.InteractiveSession()
init.run()
result=f.eval()
print(result)
sess.close()

125


In [7]:
x1=tf.Variable(1)

x1.graph is tf.get_default_graph()

True

In [8]:
tf.reset_default_graph()

In [9]:
import numpy as np
from sklearn.datasets import fetch_california_housing
# LR using Normal Equation
housing =fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]

X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_val=theta.eval()

In [10]:
print(theta_val)

[[-3.7171074e+01]
 [ 4.3633682e-01]
 [ 9.3871783e-03]
 [-1.0717344e-01]
 [ 6.4540231e-01]
 [-4.1238391e-06]
 [-3.7809242e-03]
 [-4.2373490e-01]
 [-4.3720812e-01]]


In [11]:
# LR using GD
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]

X = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),name="theta")
y_pred = tf.matmul(X,theta,name="pred")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")
# Using Manual Gradient
grad= 1/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta,theta- learning_rate*grad)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range (n_epochs):
        if epoch%100 == 0:
            print("Epoch ",epoch , "MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

    
print(best_theta)
    
    

Epoch  0 MSE= 5.653334
Epoch  100 MSE= 1.3626162
Epoch  200 MSE= 0.74454606
Epoch  300 MSE= 0.6405453
Epoch  400 MSE= 0.6138457
Epoch  500 MSE= 0.6007086
Epoch  600 MSE= 0.5910408
Epoch  700 MSE= 0.5829661
Epoch  800 MSE= 0.57600015
Epoch  900 MSE= 0.5699361
[[ 2.068468  ]
 [ 0.9218707 ]
 [ 0.1887558 ]
 [-0.34211177]
 [ 0.32638732]
 [ 0.02019763]
 [-0.04730525]
 [-0.30816892]
 [-0.28487548]]


In [12]:
# Using autodiff

grad= tf.gradients(mse,[theta])[0]
training_op = tf.assign(theta,theta- learning_rate*grad)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range (n_epochs):
        if epoch%100 == 0:
            print("Epoch ",epoch , "MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

    
print(best_theta)

Epoch  0 MSE= 9.161083
Epoch  100 MSE= 0.70065236
Epoch  200 MSE= 0.550026
Epoch  300 MSE= 0.54201794
Epoch  400 MSE= 0.538022
Epoch  500 MSE= 0.5350278
Epoch  600 MSE= 0.5327325
Epoch  700 MSE= 0.53096217
Epoch  800 MSE= 0.52958953
Epoch  900 MSE= 0.5285197
[[ 2.0685525e+00]
 [ 8.6836165e-01]
 [ 1.3765329e-01]
 [-3.1695876e-01]
 [ 3.3865386e-01]
 [ 1.8705809e-03]
 [-4.1751351e-02]
 [-7.2572112e-01]
 [-6.9983274e-01]]


In [13]:
# Using Optimizers

optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range (n_epochs):
        if epoch%100 == 0:
            print("Epoch ",epoch , "MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

    
print(best_theta)

Epoch  0 MSE= 7.5210333
Epoch  100 MSE= 0.9029567
Epoch  200 MSE= 0.71204966
Epoch  300 MSE= 0.65774345
Epoch  400 MSE= 0.6205691
Epoch  500 MSE= 0.5938169
Epoch  600 MSE= 0.5745118
Epoch  700 MSE= 0.5605766
Epoch  800 MSE= 0.5505162
Epoch  900 MSE= 0.5432523
[[ 2.0685525 ]
 [ 0.7438939 ]
 [ 0.14852081]
 [-0.01782127]
 [ 0.06315883]
 [ 0.00743982]
 [-0.04001681]
 [-0.76338315]
 [-0.7199136 ]]


In [14]:
# Placeholder

A=tf.placeholder(tf.float32,shape=(None,3))
B=A+5

with tf.Session() as sess:
    B_val_1=B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2=B.eval(feed_dict={A: [[4,5,6],[7,8,9]]})
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [15]:
X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")

batch_size=1000;
n_epochs=3
n_batches=int(np.ceil(m/batch_size))
init=tf.global_variables_initializer()
print(n_batches)
def fetch_batch (epoch ,batch_index,batch_size):
    X_batch=tf.constant(scaled_housing_data_plus_bias[batch_index*batch_size : (batch_index+1)*batch_size],dtype=tf.float32,name="X")
    y_batch=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
    return X_batch,y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            X_batch=X_batch.eval()
            y_batch=y_batch.eval()
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            print("Epoch : ",epoch*n_batches+batch_index)
        print(mse.eval())
    best_theta=theta.eval()

print(best_theta)
    
    

21
Epoch :  0
Epoch :  1
Epoch :  2
Epoch :  3
Epoch :  4
Epoch :  5
Epoch :  6
Epoch :  7
Epoch :  8
Epoch :  9
Epoch :  10
Epoch :  11
Epoch :  12
Epoch :  13
Epoch :  14
Epoch :  15
Epoch :  16
Epoch :  17
Epoch :  18
Epoch :  19
Epoch :  20
3.8434768
Epoch :  21
Epoch :  22
Epoch :  23
Epoch :  24
Epoch :  25
Epoch :  26
Epoch :  27
Epoch :  28
Epoch :  29
Epoch :  30
Epoch :  31
Epoch :  32
Epoch :  33
Epoch :  34
Epoch :  35
Epoch :  36
Epoch :  37
Epoch :  38
Epoch :  39
Epoch :  40
Epoch :  41
1.9431906
Epoch :  42
Epoch :  43
Epoch :  44
Epoch :  45
Epoch :  46
Epoch :  47
Epoch :  48
Epoch :  49
Epoch :  50
Epoch :  51
Epoch :  52
Epoch :  53
Epoch :  54
Epoch :  55
Epoch :  56
Epoch :  57
Epoch :  58
Epoch :  59
Epoch :  60
Epoch :  61
Epoch :  62
1.1983553
[[ 1.4915392 ]
 [ 0.7989615 ]
 [-0.21953104]
 [-0.70236874]
 [ 0.60696596]
 [-0.299409  ]
 [ 0.02140428]
 [-0.00430318]
 [-0.11726115]]


In [16]:
from datetime import datetime

now=datetime.utcnow().strftime("%y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

In [17]:
n_epochs=1
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            X_batch=X_batch.eval()
            y_batch=y_batch.eval()
            if batch_index % 2 == 0:
                summary_str = mse_summary.eval(feed_dict={X:X_batch,y:y_batch})
                step=epoch*n_batches+batch_index
                file_writer.add_summary(summary_str,step)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            print(step)

0
0
2
2
4
4
6
6
8
8
10
10
12
12
14
14
16
16
18
18
20


In [18]:
file_writer.close()

In [19]:
import os
os.getcwd()

'C:\\Users\\Sai\\Desktop\\Jupyter Notebooks'